This post details how to set up a multi-node, single-server Riak cluster with replication across nodes.

## Some background
- Riak operates on a server-client model: the "riak daemon" (formerly "riak start") terminal command boots up a riak daemon that runs in the background.
- Each separate running instance of riak is called a node; each Riak node manages a set of virtual nodes, or vnodes, that are responsible for storing a separate portion of the keys stored in the cluster (https://www.tiot.jp/riak-docs/riak/kv/3.2.0/learn/concepts/clusters/).
- Riak supports running two kinds of clusters:
1. One node per host
2. Multiple nodes on a single host ← (the subject of this post), (https://www.tiot.jp/riak-docs/riak/kv/3.2.0/using/running-a-cluster/#running-multiple-nodes-on-one-host)

## Key Documentation
- Python Client, http://basho.github.io/riak-python-client/ 
- Riak-KV, https://www.tiot.jp/riak-docs/riak/kv/3.2.0/ 
- Riak-KV source, https://github.com/basho/riak 

# Installation/Setup

Operating System:
CC-Ubuntu22.04

## 1. Install pyenv
Follow: https://itslinuxfoss.com/install-use-pyenv-ubuntu/


## 2. Install Python 3.5.9 and Pip
Python 3.5.9 is the latest compatible version with the Riak Python client

In [ ]:
sudo apt update
sudo apt install clang -y
sudo apt install libssl-dev
sudo apt-get install build-essential gdb lcov pkg-config \
      libbz2-dev libffi-dev libgdbm-dev libgdbm-compat-dev liblzma-dev \
      libncurses5-dev libreadline6-dev libsqlite3-dev libssl-dev \
      lzma lzma-dev tk-dev uuid-dev zlib1g-dev
CC=clang pyenv install 3.5.9
pyenv global 3.5.9

24


...Now you should be able to "import riak" in any *.py file!

## 3. Install Riak

As prereqs, we need to install Erlang OTP25, cmake, and libpam0g-dev...

### 3.a Prereqs

In [ ]:
# Install Erlang
git clone https://github.com/erlang/otp.git
cd otp
git checkout OTP-25.3.2.5
./configure
make -j9
sudo make install

# Install cmake
sudo apt-get install cmake

# Install libpam0g-dev (this fixes a pam_appl.h error when installing Riak)
sudo apt-get install libpam0g-dev

### 3.b. Install Riak proper

In [ ]:
wget https://github.com/basho/riak/archive/refs/tags/riak-3.2.0.tar.gz
tar zxvf riak-3.2.0.tar.gz
cd riak-riak-3.2.0
make devclean
make devrel

## Running a cluster
For more details: https://www.tiot.jp/riak-docs/riak/kv/3.2.0/using/running-a-cluster/#running-multiple-nodes-on-one-host 

For the following commands, the shell cwd must be in riak-riak-3.2.0.

In [ ]:
# Starting several Riak daemons (you can do up to 8 daemons, because devrel creates 8 dev* directories)
sudo dev/dev1/riak/bin/riak daemon
sudo dev/dev2/riak/bin/riak daemon
sudo dev/dev3/riak/bin/riak daemon

# Joining dev2 and dev3 to dev1 
sudo dev/dev2/riak/bin/riak admin cluster join dev1@127.0.0.1
sudo dev/dev3/riak/bin/riak admin cluster join dev1@127.0.0.1
sudo dev/dev1/riak/bin/riak admin cluster plan
sudo dev/dev1/riak/bin/riak admin cluster commit

# See members of cluster
sudo dev/dev1/riak/bin/riak admin member-status
# Note: repeat "member-status" multiple times just after commit to see "Ring" composition slowly changing to equalize across the 3 nodes

# Creating a bucket type (e.g. in the following example, we set a custom n_val replication value of 1; default is 3)
riak-admin bucket-type create n_equals_1 '{"props":{"n_val":1}}'
riak-admin bucket-type activate n_equals_1


## Interacting with cluster through Python client
...a simple python program to demonstrate replication

IMPORTANT*
Q: Where to get the "nodes" information?
A: you need to copy some fields from dev*/etc/riak.conf
- the cluster's ip address          -> host
- listener.http.internal port #     -> http_port
- listener.protobuf.internal port # -> pb_port

In [ ]:
import riak
from riak import RiakClient, RiakObject

# Creating a client connected to dev1, dev2, and dev3
# client = riak.RiakClient(protocol="pbc", 
#                          host='127.0.0.1', 
#                          nodes=[{"host":"127.0.0.1", "http_port":10018, "pb_port":10017},  # dev1
#                                 {"host":"127.0.0.1", "http_port":10028, "pb_port":10027},  # dev2
#                                 {"host":"127.0.0.1", "http_port":10038, "pb_port":10037}]) # dev3



# Creating a client connected to dev1 only
client_dev1 = riak.RiakClient(protocol="pbc", 
                         host='127.0.0.1', 
                         nodes=[{"host":"127.0.0.1", "http_port":10018, "pb_port":10017}]) # dev1


bucket = client.bucket('example_bucket')
# Note, if bucket-type is not specified, the "default" bucket-type is used (which is always available)
# You can only instantiate a bucket-type that you create beforehand!
# e.g. n_equals_1 (the bucket type we created in the previous block) is available 
# bucket = client_dev1.bucket_type("n_equals_1").bucket("r1_example0")



# Create a new object and save
wobj = bucket.new("example object")
wobj.content_type = "text/plain"  # Set the content type
wobj.data = "This is some data"
wobj.store()

robj = bucket.get("example object")
print(robj.data)


client_dev2 = riak.RiakClient(protocol="pbc", 
                              host='127.0.0.1', 
                              nodes=[{"host":"127.0.0.1", "http_port":10028, "pb_port":10027}])
print(bucket.get("example object").data)

Expected Output:
This is some data
This is some data

Explanation:
- client_dev1 is a client connected to dev1, client_dev2 to dev2
- I demonstrate that if we create and store the "Justin Shin" object in dev1, I may access it from dev2
- This shows replication of the object across a 3-node cluster
